In [1]:
path = "/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/"

import os

In [ ]:
%pip install pandas
%pip install numpy

In [2]:
import time
import glob
import pandas as pd


In [4]:

pattern = os.path.join(path,'**/*.csv')

for csv_file_paths in glob.iglob(pattern, recursive=True):
    print(csv_file_paths)

/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/Block/b8_27_eb_b8_a5_9e.csv
/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/FLSYS/fs.csv
/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/KERN/kern.csv
/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/Network/net.csv
/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/RES/res_1699894253.csv


In [5]:
import pandas as pd
import glob

df_headers_row = []
for csv_file_path in glob.iglob(pattern, recursive=True):
    temp_df = pd.read_csv(csv_file_path)
    filename = os.path.basename(csv_file_path)
    
    headers = " ".join(temp_df.columns.tolist())
    
    df_headers_row.append({'Filename': filename, 'Headers': headers})

df_headers = pd.DataFrame(df_headers_row)
df_headers

,Filename,Headers
0,b8_27_eb_b8_a5_9e.csv,timestamp read_ops write_ops read_kbs write_kb...
1,fs.csv,time timestamp seconds connectivity block:bloc...
2,kern.csv,time timestamp seconds connectivity alarmtimer...
3,net.csv,Time Protocol SourceIP SourcePort DestIP DestP...
4,res_1699894253.csv,time cpu seconds ioread iowrite ioreadbytes io...


In [6]:
import pandas as pd
import glob
from typing import Tuple, Iterator

def yield_temp_dataframes_from_csv() -> Iterator[Tuple[str, pd.DataFrame]]:
    for csv_file_paths in glob.iglob(pattern, recursive=True):
        yield os.path.basename(csv_file_paths),pd.read_csv(csv_file_paths)

In [7]:
first_rows = []
for file, df in yield_temp_dataframes_from_csv():
    data = {}
    if "timestamp" in df.columns:
        data['timestamp'] = df['timestamp'].iloc[0]
    if "time" in df.columns:
        data['time'] = df['time'].iloc[0]
    if "Time" in df.columns:
        data['Time'] = df['Time'].iloc[0]

    first_rows.append(data)

pd.DataFrame(first_rows)

,timestamp,time,Time
0,1.699896e+12,NaN,NaN
1,1.699896e+12,5.104980e+00,NaN
2,1.699896e+12,5.119650e+00,NaN
3,NaN,NaN,1.699896e+09
4,NaN,1.699896e+09,NaN


In [8]:
import numpy as np

kern_df = pd.read_csv("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/KERN/kern.csv")

# Convert the timestamp to datetime and kern timestamp is in nanoseconds
kern_df['timestamp'] = pd.to_datetime(np.floor(kern_df['timestamp'] / 1000), unit='s')
kern_df.set_index("timestamp", inplace=True, drop=False)

In [9]:
for group in kern_df.resample("20s",  origin='start'):
    window, temp_df = group
    print(window)
    print(temp_df)
    break

2023-11-13 17:24:10
                         time           timestamp   seconds  connectivity  \
timestamp                                                                   
2023-11-13 17:24:10   5.11965 2023-11-13 17:24:10  5.119650             1   
2023-11-13 17:24:17  10.37540 2023-11-13 17:24:17  5.255741             1   

                     alarmtimer:alarmtimer_fired  alarmtimer:alarmtimer_start  \
timestamp                                                                       
2023-11-13 17:24:10                            0                            0   
2023-11-13 17:24:17                            0                            0   

                     block:block_bio_backmerge  block:block_bio_remap  \
timestamp                                                               
2023-11-13 17:24:10                          7                      9   
2023-11-13 17:24:17                          0                      1   

                     block:block_dirty_buffer  block:

In [10]:
kern_df_mean = kern_df.resample("20s",  origin='start').mean()
kern_df_mean.drop(columns=['timestamp', 'time'], inplace=True)

In [11]:
kern_df_mean

,seconds,connectivity,alarmtimer:alarmtimer_fired,alarmtimer:alarmtimer_start,block:block_bio_backmerge,block:block_bio_remap,block:block_dirty_buffer,block:block_getrq,block:block_touch_buffer,block:block_unplug,...,writeback:sb_clear_inode_writeback,writeback:wbc_writepage,writeback:writeback_dirty_inode,writeback:writeback_dirty_inode_enqueue,writeback:writeback_dirty_page,writeback:writeback_mark_inode_dirty,writeback:writeback_pages_written,writeback:writeback_single_inode,writeback:writeback_write_inode,writeback:writeback_written\n
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-13 17:24:10,5.187695,1.0,0.0,0.0,3.5,5.0,3.5,1.5,205.5,0.5,...,0.0,0.0,509.5,3.5,3.5,512.0,1.0,0.0,0.0,1.0
2023-11-13 17:24:30,5.034264,1.0,0.0,0.0,1.0,4.0,0.0,3.0,5.5,0.5,...,0.5,4.0,365.5,2.5,0.0,365.0,0.5,0.5,0.0,1.0
2023-11-13 17:24:50,5.066291,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,417.0,0.0,0.0,416.0,0.0,0.0,0.0,0.0
2023-11-13 17:25:10,5.065650,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,...,0.0,0.0,527.0,0.0,0.0,527.0,0.0,0.0,0.0,0.0
2023-11-13 17:25:30,5.081419,1.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,...,0.0,0.0,498.5,0.0,0.0,498.5,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 21:22:30,5.092751,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,502.0,0.0,0.0,502.0,0.0,0.0,0.0,0.0
2023-11-13 21:22:50,5.075052,1.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,...,0.0,0.0,457.5,0.0,0.0,456.5,0.0,0.0,0.0,0.0
2023-11-13 21:23:10,5.053065,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0,...,0.0,0.0,371.5,0.0,0.0,371.0,0.0,0.0,0.0,0.0


In [12]:
def is_timestamp_in_microseconds(df:pd.DataFrame) -> bool:
    max_timestamp = df['timestamp'].max()
    return max_timestamp > time.time()

def clean_and_set_index_timestamp(df:pd.DataFrame):
    if "timestamp" in df.columns:
        pass
    elif "time" in df.columns:
        df['timestamp'] = df['time']
    elif "Time" in df.columns:
        df['timestamp'] = df['Time']
    
    if is_timestamp_in_microseconds(df):
        print("Timestamp is in microseconds")
        # df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')  
        df['timestamp'] = pd.to_datetime(np.floor(df['timestamp'] / 1000), unit='s')
    else:
        df['timestamp'] = pd.to_datetime(np.floor(df['timestamp']), unit='s')
    
    df.set_index("timestamp", inplace=True, drop=True)

    df.drop(columns=['time', 'Time', 'timestamp'], inplace=True, errors='ignore')


def create_mean_df(df:pd.DataFrame, time_window = "20s") -> pd.DataFrame:
    return df.resample(time_window,  origin='start').mean()

In [13]:
kern_df2 = pd.read_csv("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/KERN/kern.csv")

clean_and_set_index_timestamp(kern_df2)
kern_df_mean2 = create_mean_df(kern_df2)

Timestamp is in microseconds


In [14]:
for file_name, df in yield_temp_dataframes_from_csv():
    print(file_name)
    clean_and_set_index_timestamp(df)
    df_mean = create_mean_df(df)
    print(df_mean.head(1))

b8_27_eb_b8_a5_9e.csv
Timestamp is in microseconds
                     read_ops  write_ops  read_kbs  write_kbs  avgrq_sz  \
timestamp                                                                 
2023-11-13 17:24:10       0.0        9.6       0.0        2.1       0.0   

                     avg_queue  await  r_await  w_await  svctm  util  
timestamp                                                             
2023-11-13 17:24:10      83.76    0.0     22.0    0.005    0.0  28.5  
fs.csv
Timestamp is in microseconds
                      seconds  connectivity  block:block_bio_backmerge  \
timestamp                                                                
2023-11-13 17:24:10  5.104984           1.0                        7.0   

                     block:block_bio_bounce  block:block_bio_complete  \
timestamp                                                               
2023-11-13 17:24:10                     0.0                       0.0   

                     block:bloc

TypeError: agg function failed [how->mean,dtype->object]

In [100]:
net_df = pd.read_csv("/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/0_raw_collected_dataZien_device1/Network/net.csv")

clean_and_set_index_timestamp(net_df)


In [92]:
from typing import List

def filterOut_IpAddresses(df: pd.DataFrame, ip_address: List[str]) -> pd.DataFrame:
    return df[~((df["SourceIP"].isin(ip_address)) | (df["DestIP"].isin(ip_address)))]

In [101]:
net_df = filterOut_IpAddresses(net_df, ["192.168.31.34"])

In [102]:
net_df["SourceIP"].value_counts()

SourceIP
192.168.31.137    108
192.168.31.84      20
192.168.31.113     15
192.168.31.156      8
Name: count, dtype: int64

In [103]:
net_df["DestIP"].value_counts()

DestIP
224.0.0.251        73
239.255.255.250    73
224.0.0.252         3
192.168.31.255      2
Name: count, dtype: int64

In [104]:
net_df["DestPort"].value_counts()

DestPort
5353    73
1900    73
5355     3
137      2
Name: count, dtype: int64

In [105]:
net_df["SourcePort"].value_counts()[:5]

SourcePort
5353     73
43341     5
60892     4
52852     4
55457     4
Name: count, dtype: int64

In [66]:
net_df.head(1)

,Protocol,SourceIP,SourcePort,DestIP,DestPort,Length
timestamp,,,,,,
2023-11-13 17:25:12,UDP,192.168.31.84,5353,224.0.0.251,5353,139


In [60]:
def create_mean_net_df(df:pd.DataFrame, time_window = "20s") -> pd.DataFrame:
    series = []
    for timestamp, group_df in df.resample(time_window,  origin='start'):
        protocol_counts = group_df["Protocol"].value_counts()
        features = {
            "timestamp": timestamp,
            "PacketCount": group_df.shape[0],
            "TotalLength": group_df["Length"].sum(),
            "AverageLength": group_df["Length"].mean(),
            "MedianLength": group_df["Length"].median(),
            "MinLength": group_df["Length"].min(),
            "MaxLength": group_df["Length"].max(),
            "VarianceLength": group_df["Length"].var(),
            "DifferentSourcePorts": group_df["SourcePort"].nunique(),
            "DifferentDestPorts": group_df["DestPort"].nunique(),
            "TcpPacketCount": protocol_counts.get("TCP", 0),
            "UdpPacketCount": protocol_counts.get("UDP", 0),
        }
        series.append(features)
    
    return pd.DataFrame(series).set_index("timestamp", inplace=False, drop=True)


net_df_mean = create_mean_net_df(net_df)
net_df_mean

,PacketCount,TotalLength,AverageLength,MedianLength,MinLength,MaxLength,VarianceLength,DifferentSourcePorts,DifferentDestPorts,TcpPacketCount,UdpPacketCount
timestamp,,,,,,,,,,,
2023-11-13 17:25:12,1,139,139.0,139.0,139.0,139.0,NaN,1,1,0,1
2023-11-13 17:25:32,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2023-11-13 17:25:52,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2023-11-13 17:26:12,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2023-11-13 17:26:32,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 21:20:12,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2023-11-13 21:20:32,0,0,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2023-11-13 21:20:52,1,139,139.0,139.0,139.0,139.0,NaN,1,1,0,1
